# Import data

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
%matplotlib inline

found as csv on http://www.football-data.co.uk/germanym.php

In [56]:
game_mat_df = pd.read_csv('D1_2017_2018.csv', usecols=['Date','HomeTeam','AwayTeam','FTHG','FTAG'])
goals_df = game_mat_df

All team names need to be converted to a team index for numerical handling

In [57]:
team_names = game_mat_df['HomeTeam'].unique()
team_index = list(range(1,len(teams)+1))
teams_dict = dict(zip(team_names,team_index))

def team_lookup(name):
    index = teams_dict[name]
    return index

In [66]:
goals_ind_df = goals_df.copy()
goals_ind_df['HomeTeam'] = goals_df['HomeTeam'].apply(team_lookup)
goals_ind_df['AwayTeam'] = goals_df['AwayTeam'].apply(team_lookup)

## Probabilistic model